In [6]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

In [7]:
df_pop=pd.read_csv("population.csv")
df=pd.read_csv("sous_nutrition.csv")
df_dispo_A=pd.read_csv("dispo_alimentaire.csv")
df_aideA=pd.read_csv("aide_alimentaire.csv")



FileNotFoundError: [Errno 2] No such file or directory: 'population.csv'

## 1/ Proportion de personnes en etat de sous nutrition 



In [ ]:
#jointure des deux fichiers(sous_nutrition et population)
df1=df.merge(df_pop,on='Zone')

#tel que année egale 2017
df1=df1[df1["Année_x"]=="2016-2018"]
df1=df1[df1["Année_y"]==2017]
df1=df1[df1['Valeur_x']!='<0.1']

#calcul la somme sous nutrition
sum_nutrition=df1["Valeur_x"].astype(float).sum()

#calcul la somme population mondiale 2017
sum_mondial=df1["Valeur_y"].sum()
df_proportion=round(((sum_nutrition*1000000)/(sum_mondial*1000))*100,2)

print("Proportion de personnes en état de sous nutrition est de {} %".format(df_proportion))









## 2/ Nombre theorique de personnes qui pourraient être nourries (disponibilité alimentaire )

In [ ]:
#calcul somme disponibilité alimentaire parpersonne par année/personne
test1=(df_dispo_A.groupby(by=['Zone'])[['Disponibilité alimentaire (Kcal/personne/jour)']].sum()*365).reset_index()

#calcul dispo total du pays 
test1=test1.merge(df_pop,on='Zone')
test1=test1[test1["Année"]==2017]
test1["dispo"]=test1["Disponibilité alimentaire (Kcal/personne/jour)"]*test1["Valeur"]*1000

#calcul en besoin d'aliment par an/kcal/pays
test1["besoin"]=test1["Valeur"]*2500*365*1000
test1

#coefficient de disponibilité
coef=test1["dispo"].sum()/test1["besoin"].sum()
nbr_personne=round(coef*sum_mondial*1000/1e9,2)
pourcentage=round(coef*100,2)

print("Le nombre theorique de personnes qui pourraient être nourries {} milliard".format(nbr_personne),
      "soit {} % de la population mondiale".format(pourcentage))

## 3/ Nombre theorique de personnes qui pourraient être nourries (produits vegetaux )

In [ ]:
# filtre des produits vegetaux
df_vegetal=df_dispo_A[df_dispo_A["Origine"]=="vegetale"]

#calcul disponibilité alimentaire par personne par année/personne
test2=(df_vegetal.groupby(by=['Zone'])[['Disponibilité alimentaire (Kcal/personne/jour)']].sum()*365).reset_index()

#calcul dispo total du pays 
test2=test2.merge(df_pop,on='Zone')
test2=test2[test2["Année"]==2017]
test2["dispo"]=test2["Disponibilité alimentaire (Kcal/personne/jour)"]*test2["Valeur"]*1000

#calcul en besoin d'aliment par an/kcal/pays
test2["besoin"]=test2["Valeur"]*2500*365*1000

#coeficien de disponibilité
coef=test2["dispo"].sum()/test2["besoin"].sum()
nbr_pers=round(coef*sum_mondial*1000/1e9,2)
pourcentage=round(coef*100,2)
print("Le nombre theorique de personnes qui pourraient être nourries {} milliard".format(nbr_pers),
      "soit {} % de la population mondiale".format(pourcentage))





## 4/ L’utilisation de la disponibilité intérieure, en particulier la part qui est attribuée à l’alimentation animale, celle qui est perdue et celle qui est concrètement utilisée pour l'alimentation humaine.

In [ ]:
#calcul de la somme disponibilité interieur 
sum_dispo_int=df_dispo_A["Disponibilité intérieure"].sum()

#calcul disponibilité alimentaire humaine
df_dispo_b=df_dispo_A.copy().fillna(0)
df_dispo_b['Disponibilité Alimentaire humaine']=df_dispo_b['Disponibilité intérieure']-df_dispo_b['Aliments pour animaux']-df_dispo_b['Autres Utilisations']-df_dispo_b['Pertes']-df_dispo_b['Semences']-df_dispo_b['Traitement']

df_part=pd.DataFrame(columns=["Part","Pourcentage"])
for x in ["Pertes","Aliments pour animaux","Disponibilité Alimentaire humaine"]:
    somme=df_dispo_b[x].sum()
    pourcentage=round(somme/sum_dispo_int*100,2)
    df_part=df_part.append({"Part": x,
                           "Pourcentage": pourcentage},ignore_index=True)
    
df_part



## 5/ Les pays pour lesquels la proportion de personnes sous alimentées est la plus forte en 2017


In [ ]:
df_proportion=df.merge(df_pop,on='Zone')
df_proportion=df_proportion[df_proportion["Année_x"]=="2016-2018"] 
df_proportion=df_proportion[df_proportion['Année_y']==2017]
df_proportion=df_proportion[df_proportion["Valeur_x"]!='<0.1']

df_proportion['% Personnes sous-alimentées plus forte en 2017']=df_proportion['Valeur_x'].astype(float)*1e6/(df_proportion['Valeur_y']*1000)*100
df_proportion=round(df_proportion.sort_values(by='% Personnes sous-alimentées plus forte en 2017',ascending=False),1)

df_proportion[['Zone','% Personnes sous-alimentées plus forte en 2017']]




## 6/ Les pays  qui ont le plus beneficié d'aide alimentaire depuis 2013

In [ ]:
df_aideA=df_aideA[df_aideA["Année"]>=2013]
df_ben=df_aideA.groupby(['Pays bénéficiaire'])['Valeur'].sum().sort_values(ascending=False).reset_index()
df_ben=df_ben.rename(columns={"Valeur":"Aide alimentaire (en tonne)"})
df_ben



## 7/ Ceux ayant le plus/le moins de disponibilité par habitant

In [ ]:
#calcul somme disponibilité alimentaire parpersonne par jour/personne
test3=df_dispo_A.groupby(by=['Zone'])[['Disponibilité alimentaire (Kcal/personne/jour)']].sum()
test3.sort_values("Disponibilité alimentaire (Kcal/personne/jour)",ascending=False).reset_index()







